In [3]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook, reset_output, curdoc
from bokeh.embed import file_html
from bokeh.models import LabelSet, Label, ColumnDataSource, FactorRange
from bokeh.transform import factor_cmap
import bokeh.plotting as bp
from bokeh.layouts import row, column,layout
from bokeh.models.widgets import Slider, TextInput
from bokeh.transform import cumsum
from bokeh.palettes import Category10

In [4]:
data = pd.read_csv("./MC1/mc1-reports-data.csv",parse_dates=True,infer_datetime_format=True, index_col=0)
data = data.sort_index()
data.head()

,sewer_and_water,power,roads_and_bridges,medical,buildings,shake_intensity,location
time,,,,,,,
2020-04-06,2.0,4.0,1.0,NaN,4.0,NaN,17
2020-04-06,2.0,8.0,5.0,NaN,3.0,0.0,2
2020-04-06,9.0,7.0,5.0,NaN,4.0,NaN,8
2020-04-06,10.0,2.0,9.0,NaN,10.0,NaN,4
2020-04-06,6.0,2.0,9.0,NaN,4.0,1.0,15


In [5]:
data = data.fillna(0)
data.head()

,sewer_and_water,power,roads_and_bridges,medical,buildings,shake_intensity,location
time,,,,,,,
2020-04-06,2.0,4.0,1.0,0.0,4.0,0.0,17
2020-04-06,2.0,8.0,5.0,0.0,3.0,0.0,2
2020-04-06,9.0,7.0,5.0,0.0,4.0,0.0,8
2020-04-06,10.0,2.0,9.0,0.0,10.0,0.0,4
2020-04-06,6.0,2.0,9.0,0.0,4.0,1.0,15


In [4]:
data_mean = data.groupby("location").mean()
data_mean.head()

,sewer_and_water,power,roads_and_bridges,medical,buildings,shake_intensity
location,,,,,,
1,4.740072,4.759326,4.867629,4.974729,4.904934,0.329723
2,3.413544,4.313544,4.779777,0.089054,1.960111,1.266605
3,7.275434,8.461396,7.267085,7.055264,5.831991,4.590100
4,5.468654,4.224049,4.213772,0.096608,3.692018,4.263789
5,4.205263,5.065497,4.083626,2.110526,3.494737,0.519298


In [5]:
location = []
days = []
values = []
t_daño = []
for d in range(6,12):
    for l in range(1,20):
        for t in data_mean.columns:
            days.append(d)
            location.append(l)
            t_daño.append(t)
            v_tmp = data.loc[data["location"]==l].loc[data.loc[data["location"]==l].index.day==d].groupby("location").mean()[t].values
            if len(v_tmp) == 0:
                values.append(0)
            else:
                values.append(float(v_tmp))
        

In [6]:
df_pizza = pd.DataFrame()
df_pizza["day"] = days
df_pizza["values"] = np.ravel(values)
df_pizza["location"] = location
df_pizza["type"] = np.ravel(t_daño)

In [68]:
df_acu.loc[df_acu["day"]==6]

,day,values,location
0,6,105,Palace Hills
1,6,390,Northwest
2,6,596,Old Town
3,6,114,Safe Town
4,6,193,Southwest
5,6,461,Downtown
6,6,6,Wilson Forest
7,6,654,Scenic Vista
8,6,558,Broadview
9,6,206,Chapparal


In [80]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook, reset_output, curdoc
from bokeh.embed import file_html
from bokeh.models import LabelSet, Label, ColumnDataSource, FactorRange
from bokeh.transform import factor_cmap
import bokeh.plotting as bp
from bokeh.layouts import row, column,layout, gridplot
from bokeh.models.widgets import Slider, TextInput
from bokeh.transform import cumsum
from bokeh.palettes import Category10, Category20


p = figure(plot_height=480, plot_width=800, title="Danho do terremoto por dias",
              tools="pan,reset,save,wheel_zoom, hover",x_range=(30,180), y_range=(35,185))
p2 = figure(plot_height=300, plot_width=300, title="Número de chamadas por vecindario",
              tools="pan,reset,save,wheel_zoom, hover",x_range=(-4,4), y_range=(-4,4))

p3 = figure(plot_height=300, plot_width=300, title="Intensidade do terremoto por vecindario",
              tools="pan,reset,save,wheel_zoom, hover",x_range=(-4,4), y_range=(-4,4))

df_pizza = pd.read_csv("pizza/df_pizza.csv")
df_acu = pd.read_csv("pizza/data_acu.csv")
df_shake_intensity = pd.read_csv("pizza/shake_intensity.csv")
p.image_url(url=['pizza/static/mapa.svg'], x=50,y=180,w=100,h=150)  

x = np.array([65,80,100,118,80,78,148,133,110,125,135,135,120,99,88,88,110,110,98])
y = np.array([135,145,153,130,100,122,98,53,60,68,70,95,98,125,125,110,85,105,105])
l_DS = []
#CREAR TODOS LOS DATASOURCE
default_day = 6
for k in range(1,20): 
    df_p = df_pizza.loc[df_pizza["location"]==k].loc[df_pizza.loc[df_pizza["location"]==k].day==default_day]
    df_p["angle"] = df_p["values"]/(df_p["values"]).sum() * 2*np.pi
    l_DS.append(ColumnDataSource(dict(df_p, color=Category10[6])))
i=0

for idx in range(0,19):
    if idx == 0:
        p.wedge(x=x[i], y=y[i], radius=3,start_angle=cumsum('angle', include_zero=True),
                end_angle=cumsum('angle'), line_color="black", fill_color='color', source=l_DS[idx],
                legend="type", fill_alpha=.8)
    else:
        p.wedge(x=x[i], y=y[i], radius=3,start_angle=cumsum('angle', include_zero=True),
                end_angle=cumsum('angle'), line_color="black", fill_color='color', source=l_DS[idx],
               fill_alpha=.8)
    p.hover.tooltips = [
        ("name", "@type"),
        ("magnitude", "@values")
    ]
    i += 1
#PINTAR LAS LLAMADAS POR VECINDARIO
df_source = df_acu.loc[df_acu["day"]==6]
source_acu = ColumnDataSource(dict(df_source, angle=df_source["values"]/(df_source["values"]).sum() * 2*np.pi,
                                  color=Category20[19]))

p2.wedge(x=0, y=0, radius=3,start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
         line_color="black", fill_color='color', source=source_acu,fill_alpha=.8)
p2.hover.tooltips = [
        ("name", "@location"),
        ("chamadas", "@values"),
        ("dia", "@day")
    ]

#Pintar la intensidad del terremoto por vecindario
df_shake = df_shake_intensity.loc[df_shake_intensity["day"]==6]
source_shake = ColumnDataSource(dict(df_shake, angle=df_shake["values"]/(df_shake["values"]).sum() * 2*np.pi,
                                  color=Category20[19]))
p3.wedge(x=0, y=0, radius=3,start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
         line_color="black", fill_color='color', source=source_shake,fill_alpha=.8)
p3.hover.tooltips = [
        ("name", "@location"),
        ("magnitude", "@values"),
        ("dia", "@day")
    ]

#Barras de interacción
s_day_text = "registro de acordo com o dia"
s_day = Slider(title=s_day_text, value=6.0, start=6.0, end=11.0, step=1.0)
#s_magnitude = Slider(title=s_day_text, value=6.0, start=6.0, end=11.0, step=1.0)

def update_data(attrname, old, new):

    # Get the current slider values
    day_value = s_day.value
    l_aux = []
    for i in range(0,19):
        df_p1 = df_pizza.loc[df_pizza["location"]==i+1].loc[df_pizza.loc[df_pizza["location"]==i+1].day==day_value]
        df_p1["angle"] = df_p1["values"]/(df_p1["values"]).sum() * 2*np.pi
        l_DS[i].data = dict(df_p1, color=Category10[6])
    
    #para actualizar el número de llamadas acumuladas  
    df_aux = df_acu.loc[df_acu["day"]==day_value]
    source_acu.data = dict(df_aux, angle=df_aux["values"]/(df_aux["values"]).sum() * 2*np.pi,
                                  color=Category20[19])
    
    df_aux2 = df_shake_intensity.loc[df_shake_intensity["day"]==day_value]
    source_shake.data = dict(df_aux2, angle=df_aux2["values"]/(df_aux2["values"]).sum() * 2*np.pi,
                                  color=Category20[19])
for w in [s_day]:
    w.on_change('value', update_data)

# Set up layouts and add to document
inputs = column(s_day, width=200)

curdoc().add_root(row(inputs, gridplot([[p]]), column(p2,p3), width=400))

In [1]:
names = list(["Palace Hills", "Northwest","Old Town", "Safe Town","Southwest","Downtown",
                       "Wilson Forest", "Scenic Vista", "Broadview", "Chapparal", "TerrapinSprings",
                       "Pepper Mill", "CheddardFord", "Easton", "Weston", "Southon", "Oak Willow",
                       "East Parton", "West Parton"])

In [84]:
l_days = []
l_values = []
l_locations = []
for d in range(6,12):
    for l in range(1,20):
        l_days.append(d)
        l_locations.append(names[l-1])
        l_values.append((data.loc[data["location"]==l].loc[data.loc[data["location"]==l].index.day==d]).shape[0])

In [85]:
len(l_days),len(l_values), len(l_locations)

(114, 114, 114)

In [86]:
df_acu = pd.DataFrame()
df_acu["day"] = l_days
df_acu["values"] = l_values
df_acu["location"] = l_locations
df_acu.head()

,day,values,location
0,6,105,Palace Hills
1,6,390,Northwest
2,6,596,Old Town
3,6,114,Safe Town
4,6,193,Southwest


In [87]:
df_acu.to_csv("pizza/data_acu.csv", index=False)

In [18]:
l_day = []
l_loc = []
l_mag = []
for d in range(6,12): #recorre cada día
    for l in range(1,20): #recorre cada vecindario}
        l_day.append(d)
        l_loc.append(names[l-1])
        l_mag.append(data.loc[data["location"]==l].loc[data.loc[data["location"]==l].index.day==d]["shake_intensity"].mean())

In [19]:
len(l_day), len(l_loc), len(l_mag)

(114, 114, 114)

In [20]:
df_shake_intensity = pd.DataFrame()
df_shake_intensity["day"] = l_day
df_shake_intensity["location"] = l_loc
df_shake_intensity["values"] = l_mag
df_shake_intensity.head()

,day,location,values
0,6,Palace Hills,0.133333
1,6,Northwest,0.148718
2,6,Old Town,0.451342
3,6,Safe Town,0.684211
4,6,Southwest,0.191710


In [21]:
df_shake_intensity.to_csv("./pizza/shake_intensity.csv", index=False)